In [1]:
from servers.infer import extract_html, preprocess_title_and_lead, preprocess_content, summarize
from train_ly.data import get_data
from tqdm import tqdm
import pandas as pd

/root/miniconda3/envs/podcast_summarization/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/servers/podcast_summarization


In [2]:
import py_vncorenlp
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/servers/podcast_summarization/vncorenlp')

2023-01-09 11:12:17 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [3]:
def preprocess_data(title='', lead='', content=''):
    if title != '':
        title = " ".join(rdrsegmenter.word_segment(title))
        title = title + " . "
    if lead != '':
        lead = " ".join(rdrsegmenter.word_segment(lead))
    if content != '':
        content = " ".join(rdrsegmenter.word_segment(content))
    document =  title + lead + " " + content
    return document

In [4]:
fr = ['5', '1', '2023']
to = ['9', '1', '2023']
data = get_data(fr,to)
# urls = [dic['new_url'] for dic in data]
# btv = [dic['content_btv'] for dic in data]

https://ss.vnexpress.net/api/list?from_date=5%2F1%2F2023&to_date=9%2F1%2F2023


In [7]:
def run(url):
    title, lead, content = extract_html(url)
    title = preprocess_title_and_lead(title)
    lead = preprocess_title_and_lead(lead)
    content = preprocess_content(content)
    
    docment_origin = preprocess_data(title=title, lead=lead, content=content)
    docment_new = preprocess_data(title='', lead='', content=content)
    output_origin, _, _ = summarize(docment_origin)
    output_new, _, _ = summarize(docment_new)
    
    output_new = lead + '. ' + output_new
    return output_origin, output_new

In [16]:
origin = []
new = []
btvs = []
urls = []
for dic in tqdm(data):
    try:
        url = dic['new_url']
        btv = dic['content_btv']
        output_origin, output_new = run(url)
        origin.append(output_origin)
        new.append(output_new)
        btvs.append(btv)
        urls.append(url)
    except:
        print(url)

 56%|████████████████████████████████████████████████████████████████████████▉                                                          | 49/88 [02:59<02:11,  3.38s/it]

https://vnexpress.net/indonesia-vs-viet-nam-4557023-tong-thuat.html


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [05:19<00:00,  3.63s/it]


In [19]:
df = pd.DataFrame({'url': urls, 'origin': origin, 'new': new, 'btv': btvs})

In [20]:
df

,url,origin,new,btv
0,https://vnexpress.net/anh-buoc-trum-buon-nguoi...,"Toà án Anh vừa yêu cầu Ronan Hughes, kẻ cầm đầ...","Toà án Anh yêu cầu Ronan Hughes, kẻ cầm đầu đư...","Tòa án Anh yêu cầu Ronan Hughes, kẻ cầm đầu đư..."
1,https://vnexpress.net/so-nguoi-chet-do-covid-1...,Nhật Bản vừa gióng lên hồi chuông cảnh báo vì ...,Nhật Bản gióng lên hồi chuông cảnh báo vì số c...,Số người chết do Covid-19 ở Nhật Bản đột ngột ...
2,https://vnexpress.net/bao-loan-o-nha-quoc-hoi-...,Những người ủng hộ cựu lãnh đạo Brazil Jair Bo...,Những người ủng hộ cựu lãnh đạo Brazil Jair Bo...,"Hôm qua, những người ủng hộ cựu lãnh đạo Brazi..."
3,https://vnexpress.net/kiev-noi-ukraine-la-bai-...,Ukraine nói chiến sự với Nga khiến nước này hi...,Thủ tướng Ukraine Denys Shmyhal nói chiến sự v...,Thủ tướng Ukraine nói chiến sự với Nga khiến n...
4,https://vnexpress.net/ukraine-bac-tin-nga-ha-h...,Ukraine bác tin Nga hạ hơn 600 binh sĩ nước nà...,Ukraine bác tin Nga hạ hơn 600 binh sĩ nước nà...,Sau khi Nga tuyên bố hạ hơn 600 binh sĩ Ukr...
...,...,...,...,...
82,https://vnexpress.net/ong-macron-hua-chuyen-xe...,Tổng thống Pháp Macron vừa đồng ý cung cấp cho...,Ông Macron đồng ý cung cấp cho Kiev một số xe ...,Tổng thống Pháp đồng ý cung cấp cho Kiev một s...
83,https://vnexpress.net/ong-vo-van-thuong-xu-ly-...,Ông Võ Văn Thưởng đề nghị Ban Nội chính Trung ...,Thường trực Ban Bí thư Võ Văn Thưởng đề nghị B...,Thường trực Ban Bí thư Võ Văn Thưởng đề nghị B...
84,https://vnexpress.net/4-pho-giam-doc-trung-tam...,4 phó giám đốc Trung tâm đăng kiểm xe cơ giới ...,4 phó giám đốc Trung tâm đăng kiểm xe cơ giới ...,4 phó giám đốc trung tâm đăng kiểm 9 8 0 6 D t...
85,https://vnexpress.net/ong-trump-keu-goi-dang-c...,Cựu tổng thống Mỹ Trump vừa kêu gọi các đảng v...,Cựu tổng thống Mỹ Trump kêu gọi các đảng viên ...,Cựu tổng thống Mỹ Donald Trump mới đây kêu gọi...


In [21]:
df.to_excel('/servers/podcast_summarization/result.xlsx', index=False)

In [42]:
!pwd

/servers/podcast_summarization/vncorenlp
